<a href="https://colab.research.google.com/github/leul12/Data_Exploration/blob/main/tweet_stalking_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import itertools
from datetime import datetime
import datetime
import time
from pandas.io.json import json_normalize
import json
# from tqdm.notebook import tqdm_notebook
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

## Let's Check the version 🐍

In [2]:
from platform import python_version
print(python_version())

3.10.4


## Okay now we can do our **job**

1.   Let's install the 🔨
2.   Extract the 🗄️
3.   And So on



# ⬆️ install the tool 🔨

In [3]:
pip install git+https://github.com/JustAnotherArchivist/snscrape.git

  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-4uxtjp28
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-4uxtjp28
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit b515a66b937276eafff41cf6b897c53ecf3d54a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


## 🤔   **how to extract the tweets ?** no worries ⬇️

In [4]:
import snscrape.modules.twitter as sntwitter

# okay now some magic to get the actual 🐦 using the **itertools** 😉

In [5]:
df = pd.DataFrame(itertools.islice(
    sntwitter.TwitterSearchScraper('#vaccine').get_items(), 300))
# let's Test what kind of data we can get from it (if it works)
df.head(2)

,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,https://twitter.com/itzpms/status/161822092993...,2023-01-25 12:15:09+00:00,Then WHAT does the #vaccine do https://t.co/1x...,Then WHAT does the #vaccine do https://t.co/1x...,1618220929938587648,"{'username': 'itzpms', 'id': 60568748, 'displa...",0,0,0,0,...,NaN,None,None,None,None,[vaccine],None,None,NaN,None
1,https://twitter.com/CapeCodGiftShop/status/161...,2023-01-25 12:14:54+00:00,Why get a #vaccine that is in the news stating...,Why get a #vaccine that is in the news stating...,1618220865291956224,"{'username': 'CapeCodGiftShop', 'id': 66113544...",1,0,0,0,...,NaN,None,None,None,None,"[vaccine, vaccinegenocide, VaccineSideEffects]",None,{'title': 'Survey of previously vaccinated US ...,3.0,None


# okay Now that we got this out of the way what now 🤔


*   we check data heads
*   clean some columns [user , hastags , date , mentionedUsers & inReplyToUser ]
*   and take it form there 
*   🚶‍♂️



In [6]:
dfuser= pd.json_normalize(df['user'])
dfuser.head(2)

,username,id,displayname,rawDescription,renderedDescription,descriptionLinks,verified,created,followersCount,friendsCount,...,location,protected,profileImageUrl,profileBannerUrl,label,link.text,link.url,link.tcourl,link.indices,link
0,itzpms,60568748,Red Elph,get educated! if you’ve never lived in a Commu...,get educated! if you’ve never lived in a Commu...,None,False,2009-07-27 11:24:31+00:00,1452,2364,...,Right Where I Am,False,https://pbs.twimg.com/profile_images/160456331...,https://pbs.twimg.com/profile_banners/60568748...,None,pms2go.com,http://www.pms2go.com,https://t.co/PmQGallme7,"(0, 23)",NaN
1,CapeCodGiftShop,66113544,Spirit Baker,"eBay, Mercari, RedBubble Buyer and Seller - h...","eBay, Mercari, RedBubble Buyer and Seller - r...","[{'text': 'redbubble.com/people/CapeCod…', 'ur...",False,2009-08-16 13:19:04+00:00,15629,8868,...,United States Dromomaniac,False,https://pbs.twimg.com/profile_images/145962121...,https://pbs.twimg.com/profile_banners/66113544...,None,NaN,NaN,NaN,NaN,NaN


# NOw Let's Take what we need and add the to the main Data Frame ⚙️

In [7]:
dfuser.drop(['id','displayname','rawDescription','renderedDescription','descriptionLinks','location','statusesCount','favouritesCount','listedCount',
                  'mediaCount','link','profileImageUrl','profileBannerUrl','label','link.text','link.url','link.tcourl','link.indices'], axis = 1, inplace = True)
dfuser.head(2)

,username,verified,created,followersCount,friendsCount,protected
0,itzpms,False,2009-07-27 11:24:31+00:00,1452,2364,False
1,CapeCodGiftShop,False,2009-08-16 13:19:04+00:00,15629,8868,False


# now that we 🫧 the 🧫  time to add it to the clean data sets

In [8]:
df2=df.drop(['user','renderedContent','sourceUrl','hashtags','cashtags','card','inReplyToTweetId','inReplyToUser','mentionedUsers','coordinates','links','media','retweetedTweet','quotedTweet','place'],axis=1)
df2.head(2)

,url,date,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceLabel,viewCount,vibe
0,https://twitter.com/itzpms/status/161822092993...,2023-01-25 12:15:09+00:00,Then WHAT does the #vaccine do https://t.co/1x...,1618220929938587648,0,0,0,0,1618220929938587648,en,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,NaN,None
1,https://twitter.com/CapeCodGiftShop/status/161...,2023-01-25 12:14:54+00:00,Why get a #vaccine that is in the news stating...,1618220865291956224,1,0,0,0,1618220865291956224,en,"<a href=""http://twitter.com/download/android"" ...",Twitter for Android,3.0,None


In [25]:
res = pd.concat([df2, dfuser], axis=1)
res.head(2)

,url,date,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceLabel,viewCount,vibe,username,verified,created,followersCount,friendsCount,protected
0,https://twitter.com/itzpms/status/161822092993...,2023-01-25 12:15:09+00:00,Then WHAT does the #vaccine do https://t.co/1x...,1618220929938587648,0,0,0,0,1618220929938587648,en,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,NaN,None,itzpms,False,2009-07-27 11:24:31+00:00,1452,2364,False
1,https://twitter.com/CapeCodGiftShop/status/161...,2023-01-25 12:14:54+00:00,Why get a #vaccine that is in the news stating...,1618220865291956224,1,0,0,0,1618220865291956224,en,"<a href=""http://twitter.com/download/android"" ...",Twitter for Android,3.0,None,CapeCodGiftShop,False,2009-08-16 13:19:04+00:00,15629,8868,False


# ⏱️ manuplation

In [26]:
res.sort_values('date')
res.head(2)

,url,date,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceLabel,viewCount,vibe,username,verified,created,followersCount,friendsCount,protected
0,https://twitter.com/itzpms/status/161822092993...,2023-01-25 12:15:09+00:00,Then WHAT does the #vaccine do https://t.co/1x...,1618220929938587648,0,0,0,0,1618220929938587648,en,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,NaN,None,itzpms,False,2009-07-27 11:24:31+00:00,1452,2364,False
1,https://twitter.com/CapeCodGiftShop/status/161...,2023-01-25 12:14:54+00:00,Why get a #vaccine that is in the news stating...,1618220865291956224,1,0,0,0,1618220865291956224,en,"<a href=""http://twitter.com/download/android"" ...",Twitter for Android,3.0,None,CapeCodGiftShop,False,2009-08-16 13:19:04+00:00,15629,8868,False


#### 🔀 the columns

In [27]:
def swap_columns(result, col1, col2):
    col_list = list(result.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    result = result[col_list]
    return result
res = swap_columns(res,'url','date')
res.head(5)

,date,url,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceLabel,viewCount,vibe,username,verified,created,followersCount,friendsCount,protected
0,2023-01-25 12:15:09+00:00,https://twitter.com/itzpms/status/161822092993...,Then WHAT does the #vaccine do https://t.co/1x...,1618220929938587648,0,0,0,0,1618220929938587648,en,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,NaN,None,itzpms,False,2009-07-27 11:24:31+00:00,1452,2364,False
1,2023-01-25 12:14:54+00:00,https://twitter.com/CapeCodGiftShop/status/161...,Why get a #vaccine that is in the news stating...,1618220865291956224,1,0,0,0,1618220865291956224,en,"<a href=""http://twitter.com/download/android"" ...",Twitter for Android,3.0,None,CapeCodGiftShop,False,2009-08-16 13:19:04+00:00,15629,8868,False
2,2023-01-25 12:14:26+00:00,https://twitter.com/itzpms/status/161822074869...,#vaccine #diedsuddenly #covid https://t.co/vRw...,1618220748698533891,0,0,0,0,1618220748698533891,qme,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,NaN,None,itzpms,False,2009-07-27 11:24:31+00:00,1452,2364,False
3,2023-01-25 12:13:37+00:00,https://twitter.com/JustinaElumeze/status/1618...,#ToldYouSo…\n\nHope my ex female colleagues at...,1618220543538499586,0,0,0,0,1618220543538499586,en,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,NaN,None,JustinaElumeze,False,2011-06-23 14:38:13+00:00,1117,570,False
4,2023-01-25 12:13:28+00:00,https://twitter.com/DanChesler/status/16182205...,#vaccine #cardiac #injury #covid #mrna #advers...,1618220505827540994,0,0,0,0,1618220505827540994,qht,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,1.0,None,DanChesler,False,2012-05-22 21:31:57+00:00,676,3355,False


#### ⌚ to ⚠️ and 📅 

In [28]:
res['date'] = pd.to_datetime(res['date'])
datetime_col = ['date']
def extract_date_info(df, col,):
    for feat in col:
        res[feat +'_day'] = res[feat].dt.day
        res[feat +'_month'] = res[feat].dt.month
        res[feat+'_WeekDay'] = res[feat].dt.weekday
        res[feat+'_Quarter'] = res[feat].dt.quarter
        res[feat+'_year'] = res[feat].dt.year
        res['Tweet time'] = res[feat].dt.time
        res['monthname'] = res[feat].dt.month_name()
        res['dayname'] = res[feat].dt.day_name()
    res.drop(columns=col,axis=1,inplace=True)

extract_date_info(res, datetime_col)
res

,url,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,...,friendsCount,protected,date_day,date_month,date_WeekDay,date_Quarter,date_year,Tweet time,monthname,dayname
0,https://twitter.com/itzpms/status/161822092993...,Then WHAT does the #vaccine do https://t.co/1x...,1618220929938587648,0,0,0,0,1618220929938587648,en,"<a href=""http://twitter.com/download/iphone"" r...",...,2364,False,25,1,2,1,2023,12:15:09,January,Wednesday
1,https://twitter.com/CapeCodGiftShop/status/161...,Why get a #vaccine that is in the news stating...,1618220865291956224,1,0,0,0,1618220865291956224,en,"<a href=""http://twitter.com/download/android"" ...",...,8868,False,25,1,2,1,2023,12:14:54,January,Wednesday
2,https://twitter.com/itzpms/status/161822074869...,#vaccine #diedsuddenly #covid https://t.co/vRw...,1618220748698533891,0,0,0,0,1618220748698533891,qme,"<a href=""http://twitter.com/download/iphone"" r...",...,2364,False,25,1,2,1,2023,12:14:26,January,Wednesday
3,https://twitter.com/JustinaElumeze/status/1618...,#ToldYouSo…\n\nHope my ex female colleagues at...,1618220543538499586,0,0,0,0,1618220543538499586,en,"<a href=""http://twitter.com/download/iphone"" r...",...,570,False,25,1,2,1,2023,12:13:37,January,Wednesday
4,https://twitter.com/DanChesler/status/16182205...,#vaccine #cardiac #injury #covid #mrna #advers...,1618220505827540994,0,0,0,0,1618220505827540994,qht,"<a href=""https://mobile.twitter.com"" rel=""nofo...",...,3355,False,25,1,2,1,2023,12:13:28,January,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,https://twitter.com/Meeno17043089/status/16180...,Biological weapon expert #StevenHatfill:\nAcco...,1618002548715323397,0,0,0,0,1618002548715323397,en,"<a href=""http://twitter.com/download/android"" ...",...,546,False,24,1,1,1,2023,21:47:23,January,Tuesday
296,https://twitter.com/Zed_Blockchain/status/1618...,@Censored4sure @HamlinIsland @Censored4sure \n...,1618001867216420867,0,0,6,0,1617968370800013313,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",...,2676,False,24,1,1,1,2023,21:44:40,January,Tuesday
297,https://twitter.com/Zed_Blockchain/status/1618...,@Stephen72925359 \n\n#DamarHamlin has to under...,1618001524805996544,0,0,1,0,1618001524805996544,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",...,2676,False,24,1,1,1,2023,21:43:19,January,Tuesday
298,https://twitter.com/Zed_Blockchain/status/1618...,#DamarHamlin has to understand that his situat...,1618001049360699394,0,0,2,1,1618001049360699394,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",...,2676,False,24,1,1,1,2023,21:41:25,January,Tuesday
